In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


%run nb_init.py
# This imports leg_joint as lj


In [ ]:
lj_eptm = lj.Epithelium(lj.data.small_xml(), save_dir='/tmp', copy=True)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline

import tyssue as ty
from tyssue.core.sheet import Sheet
from tyssue.geometry.sheet_geometry import SheetGeometry  as geom
from tyssue.config.json_parser import load_default

In [ ]:
graph = lj_eptm.graph
print('****** Vertex ********')
print(*graph.vertex_properties.keys(), sep='\n')
print('****** Edge ********')

print(*graph.edge_properties.keys(), sep='\n')

      

In [ ]:
specs = load_default('geometry', 'sheet')

print('CELLS')
print(*specs['face'].keys(), sep='\t')

print('VERTICES')
print(*specs['vert'].keys(), sep='\t')

print('EDGES')
print(*specs['edge'].keys(), sep='\t')

In [ ]:
vert_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z', 'rhos':'rho',
               'is_active_vert': 'is_active'}

cell_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z',
               'areas': 'area', 'perimeters': 'perimeter',
               'is_alive': 'is_alive'}

edge_import = {'dixs': 'dx', 'dwys': 'dy', 'dzeds': 'dz',
               'edge_lengths': 'length'}

In [ ]:
def get_faces(graph, as_array=True):
    '''
    Retrieves all the triangular subgraphs of the form
       1 -- > 2
        ^   ^
         \ /
          0
    In our context, vertex 0 always corresponds to a cell/face
    and vertices 1 and 2 to junction vertices

    Parameters
    ----------
    graph : a :class:`GraphTool` graph instance
    as_array: bool, optional, default `True`
      if `True`, the output of `subraph_isomorphism` is converted
      to a (N, 3) ndarray.

    Returns
    -------
    triangles:  list of gt.PropertyMaps or (N, 3) ndarray
      each line corresponds to a triplet (cell, jv0, jv1)
      where cell, jv0 and jv1 are indices of the input graph.
    '''
    tri_graph = gt.Graph()
    ## the vertices
    verts = tri_graph.add_vertex(3)
    ## edges
    tri_graph.add_edge_list([(0, 1), (0, 2), (1, 2)])
    _triangles = gt.subgraph_isomorphism(tri_graph, graph)
    if not as_array:
        return tri_graph, _triangles
    triangles = np.array([tri.a for tri in _triangles], dtype=np.int)
    return triangles

In [ ]:
triangles = get_faces(graph)

The new base triangle is oriented differently, as the edge is splited.

We now have:

```
       1 -- > 2             2 <-- 1  
        ^   /                ^   /
         \ v     and          \ v  
          0                    0
```



In [ ]:
pos = gt.group_vector_property([lj_eptm.ixs, lj_eptm.wys, lj_eptm.zeds])
deltas = gt.group_vector_property([graph.ep['dixs'], graph.ep['dwys'],  graph.ep['dzeds']])

oriented = []
for (cell, jv0, jv1) in triangles:
    
    normal = np.cross(deltas[graph.edge(cell, jv0)],
                      deltas[graph.edge(cell, jv1)])
    orientation = np.sign(np.sum(pos[graph.vertex(cell)] * normal))
    if orientation > 0:
        oriented.append([jv0, jv1, cell])
    else:
        oriented.append([jv1, jv0, cell])
    
    

In [ ]:
graph.set_vertex_filter(lj_eptm.is_cell_vert)
cells_idx_in = graph.vertex_index.copy()
cells_idx_in = cells_idx_in.fa

graph.set_vertex_filter(lj_eptm.is_cell_vert, True)
jv_idx_in = graph.vertex_index.copy()
jv_idx_in = jv_idx_in.fa
graph.set_vertex_filter(None)



In [ ]:
face_df = pd.DataFrame(index=pd.Index(range(cells_idx_in.shape[0]), name='face'), 
                       columns=cell_import.values())
face_df['old_idx'] = cells_idx_in

for data_in, data_out in cell_import.items():
    face_df[data_out] = graph.vp[data_in].a[cells_idx_in]

vert_df = pd.DataFrame(index=pd.Index(range(jv_idx_in.shape[0]), name='vert'), 
                       columns=vert_import.values())
vert_df['old_idx'] = jv_idx_in
    
    
for data_in, data_out in vert_import.items():
    vert_df[data_out] = graph.vp[data_in].a[jv_idx_in]



In [ ]:
re_indexed = []
for jv0, jv1, face in oriented:
    srce = np.where(vert_df.old_idx==jv0)[0][0]
    trgt = np.where(vert_df.old_idx==jv1)[0][0]
    r_face = np.where(face_df.old_idx==face)[0][0]
    re_indexed.append((srce, trgt, r_face))
re_indexed = np.array(re_indexed)

In [ ]:
re_indexed.shape

In [ ]:
edge_idx = pd.Index(np.arange(re_indexed.shape[0]), name='edge')

edge_df = pd.DataFrame(index=edge_idx, columns=edge_import.values())
edge_df['srce'] = re_indexed[:, 0]
edge_df['trgt'] = re_indexed[:, 1]
edge_df['face'] = re_indexed[:, 2]


edge_df['old_jv0'] = triangles[:, 1]
edge_df['old_jv1'] = triangles[:, 2]
edge_df['old_cell'] = triangles[:, 0]

eps = gt.group_vector_property([graph.ep[col] for col in edge_import.keys()])

In [ ]:
normal = np.cross(deltas[graph.edge(0, 56)],
                  deltas[graph.edge(0, 57)])
orientation = np.sign(np.sum(pos[graph.vertex(0)] * normal))
print(orientation)

In [ ]:
edge_df.head()


In [ ]:
datasets = {'edge': edge_df, 'vert': vert_df, 'face': face_df}

In [ ]:
sheet = Sheet('imported', datasets, specs)

In [ ]:
sheet.set_geom('sheet')

In [ ]:
geom.update_all(sheet)

In [ ]:
sheet.vert_df.head()

In [ ]:
rm small_hexagonal.hf5

In [ ]:
with pd.get_store('small_hexagonal.hf5') as store:
    store.put('face', sheet.face_df, format='t', dropna=False)
    store.put('vert', sheet.vert_df, format='t', dropna=False)
    store.put('edge', sheet.edge_df, format='t', dropna=False)

In [ ]:
from tyssue.draw.mpl_draw import sheet_view


In [ ]:
fig, ax = sheet_view(sheet, coords=['z', 'x'])